In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from PIL import Image, ImageStat

In [2]:
wb = load_workbook('Послойное описание с фотографиями.xlsx')
ws = wb[wb.sheetnames[0]]
df = pd.DataFrame(ws.values).loc[4:56,14:17]
df = df.rename(columns={14:'Порода', 15:'Карбонатность', 16:'Разрушенность', 17:'Насыщение'})

In [3]:
pics = {}
path = 'pics/'

for row in df.itertuples():
    elems = []
    for elem in os.listdir(path+str(row[0]-3)):
        elems.append(path+str(row[0]-3)+'/'+elem)
    src = elems[:int(len(elems)/2)]
    uv = elems[int(len(elems)/2):]
    src = np.concatenate([cv2.imread(img,1) for img in src], axis=0)
    uv = np.concatenate([cv2.imread(img,1) for img in uv], axis=0)
    pics[row[0]] = [ cv2.cvtColor(src, cv2.COLOR_BGR2RGB), 
                    cv2.cvtColor(uv, cv2.COLOR_BGR2RGB) ]

In [4]:
def get_oil_features(imgs):
    oil_features = []
    
    for img in imgs:
        pil_img = Image.fromarray(img)
        bright = sum(ImageStat.Stat(pil_img).mean) // 3
        dark = (85*(bright/85), 100*(bright/85), 100*(bright/85))
        light = (255, 255, 255)
        mask_yel = cv2.inRange(img, dark, light)
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_yel = cv2.bitwise_and(img_gray, img_gray, mask=mask_yel)
        pil_img_yel = Image.fromarray(img_yel)
        size = len(img) * len(img[0])
        area = sum([sum([0 if pxl == 0 else 1 for pxl in row])for row in img_yel])
        feature = {}
        #feature['size'] = len(img) * len(img[0])
        feature['area'] = area / size
        #feature['bright'] = bright
        feature['mean'] = sum([sum([pxl for pxl in row]) for row in img_yel]) / (area*bright)
        oil_features.append(feature)
    
    return oil_features

In [5]:
oil_features = get_oil_features([value[1] for value in pics.values()])
df['oil_features'] = oil_features
df.head(53)

,Порода,Карбонатность,Разрушенность,Насыщение,oil_features
4,песчаник,не карбонатный,не разрушен,нефтенасыщенные,"{'area': 0.24291390728476822, 'mean': 1.990411..."
5,алевролит,не карбонатный,не разрушен,не опред.,"{'area': 0.13807339449541284, 'mean': 2.623168..."
6,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,"{'area': 0.2994583333333333, 'mean': 1.7854128..."
7,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,"{'area': 0.24625, 'mean': 2.057161774442728}"
8,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,"{'area': 0.25641221374045803, 'mean': 1.507021..."
9,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,"{'area': 0.2885714285714286, 'mean': 2.0340936..."
10,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,"{'area': 0.12838709677419355, 'mean': 1.508317..."
11,алевролит,не карбонатный,не разрушен,не опред.,"{'area': 0.33168, 'mean': 1.8062738352037426}"
12,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,"{'area': 0.20016129032258065, 'mean': 1.587312..."
13,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,"{'area': 0.24846153846153846, 'mean': 1.873241..."


In [6]:
X = [list(feature.values()) for feature in df['oil_features']]
y = [0. if oil=='не опред.' else 1. for oil in df['Насыщение']]

In [7]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import ShuffleSplit, cross_val_score

cv = ShuffleSplit(n_splits=50, test_size=0.3, random_state=0)

In [8]:
def score(model, X, y, cv):
    scores = cross_val_score(model, X, y,
                              cv=cv,
                              scoring=make_scorer(accuracy_score))
    return scores.mean()

In [9]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=10, 
                         learning_rate=1, random_state=0)
print('AdaBoost: ', score(ada, X, y, cv))

rfc = RandomForestClassifier(n_estimators=10, 
                             random_state=0)
print('RandomForestClassifier: ', score(rfc, X, y, cv))

gbc = GradientBoostingClassifier(n_estimators=10, random_state=0)
print('GradientBoostingClassifier: ', score(gbc, X, y, cv))

AdaBoost:  0.86625
RandomForestClassifier:  0.875
GradientBoostingClassifier:  0.865


In [10]:
print(ada.fit(X,y).feature_importances_)
print(rfc.fit(X,y).feature_importances_)
print(gbc.fit(X,y).feature_importances_)

[0.31718013 0.68281987]
[0.46691348 0.53308652]
[0.70964624 0.29035376]
